# Pytorch Course
## Softmax and Crossentropy with MNIST

In [31]:
import torch
import torch.utils.data
import torch.nn.functional as F

import torchvision.datasets as datasets
import torchvision.transforms as transforms

device = torch.device("cuda:0") 

# Traininig settings
batch_size = 64
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', 
                   train=True, 
                   download=True, 
                   transform=transforms.Compose([
                       transforms.ToTensor(), 
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
                  batch_size=batch_size,
                  pin_memory=True,
                  shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', 
                   train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(), 
                       transforms.Normalize((0.1307,), (0.3081,))                       
                   ])),
                  pin_memory=True,
                  batch_size=batch_size,
                  shuffle=True
)

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.l1 = torch.nn.Linear(784, 520)
        self.l2 = torch.nn.Linear(520, 320)
        self.l3 = torch.nn.Linear(320, 240)
        self.l4 = torch.nn.Linear(240, 120)
        self.l5 = torch.nn.Linear(120, 10)
        
    def forward(self, x):
        x = x.view(-1, 784) #Flatten the datqa (n, 1, 28, 28) -> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return F.relu(self.l5(x))

model = Model()

criterion = torch.nn.CrossEntropyLoss(reduction='elementwise_mean') # Loss Function
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5) # Reduction Loss Function
     
   

In [35]:
# Training Loop
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print("Train epoch: {} [{}/{} ({: .0f}%)]\tLoss: {:.6f}".format(
                epoch, 
                batch_idx * len(data),
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item()
            ))
    

In [36]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max log-probability
        pred = torch.max(output.data, 1)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    test_loss /= len(test_loader.dataset)
    print("\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
        test_loss,
        correct,
        len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
        

In [37]:
Y_pred_scores = torch.Tensor([[0.3, 0.2, 0.9, 0.1]])

# torch.max returns both the max values and indices
Y_pred_val, Y_pred_idx = torch.max(Y_pred_scores.data, 1)

In [ ]:
for epoch in range(1, 20):
    train(epoch)
    test()

Train epoch: 1 [0/60000 ( 0%)]	Loss: 2.179891
Train epoch: 1 [6400/60000 ( 11%)]	Loss: 2.045934
Train epoch: 1 [12800/60000 ( 21%)]	Loss: 1.825020
Train epoch: 1 [19200/60000 ( 32%)]	Loss: 1.392467
Train epoch: 1 [25600/60000 ( 43%)]	Loss: 1.497786
Train epoch: 1 [32000/60000 ( 53%)]	Loss: 1.520425
Train epoch: 1 [38400/60000 ( 64%)]	Loss: 1.108231
Train epoch: 1 [44800/60000 ( 75%)]	Loss: 0.909435
Train epoch: 1 [51200/60000 ( 85%)]	Loss: 0.830704
Train epoch: 1 [57600/60000 ( 96%)]	Loss: 0.755785

Test set: Average loss: 0.0114, Accuracy: 7743/10000 (77%)

Train epoch: 2 [0/60000 ( 0%)]	Loss: 0.802100
Train epoch: 2 [6400/60000 ( 11%)]	Loss: 0.841226
Train epoch: 2 [12800/60000 ( 21%)]	Loss: 0.271182
Train epoch: 2 [19200/60000 ( 32%)]	Loss: 0.526697
Train epoch: 2 [25600/60000 ( 43%)]	Loss: 0.332947
Train epoch: 2 [32000/60000 ( 53%)]	Loss: 0.603698
Train epoch: 2 [38400/60000 ( 64%)]	Loss: 0.601210
Train epoch: 2 [44800/60000 ( 75%)]	Loss: 0.319190
Train epoch: 2 [51200/60000 ( 85%